## 评估模型

In [1]:
import sys
import os
os.chdir("/root/private_data/luog/AbAgker")

import json
import yaml
import torch
from omegaconf import OmegaConf
from taming.models.model_pmllm_S1 import PmllmModel


DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def load_config(config_path, display=False):
  config = OmegaConf.load(config_path)
  if display:
    print(yaml.dump(OmegaConf.to_container(config)))
  return config

def load_model(config, ckpt_path=None):
    model = PmllmModel(**config.model.params)
    if ckpt_path is not None:
        model.init_from_ckpt(ckpt_path)
    return model.eval()


config_path="/root/private_data/luog/AbAgker/logs/2025-02-19T20-16-59_M1_esm2_palm_dataflod/configs/2025-02-19T20-16-59-project.yaml"
ckpt_path="/root/private_data/luog/AbAgker/logs/2025-02-19T20-16-59_M1_esm2_palm_dataflod/checkpoints/last.ckpt"

config=load_config(config_path)
model=load_model(config, ckpt_path)
model = model.to(DEVICE)

Some weights of EsmModel were not initialized from the model checkpoint at /root/private_data/luog/pretrained_LLM/esm2_t30 and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.8/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028


Loading pre-trained tokenizer...
Restored from /root/private_data/luog/AbAgker/logs/2025-02-19T20-16-59_M1_esm2_palm_dataflod/checkpoints/last.ckpt


In [2]:
import argparse, os, sys, datetime, glob, importlib

def get_obj_from_str(string, reload=False):
    module, cls = string.rsplit(".", 1)
    if reload:
        module_imp = importlib.import_module(module)
        importlib.reload(module_imp)
    return getattr(importlib.import_module(module, package=None), cls)

def instantiate_from_config(config):
    if not "target" in config:
        raise KeyError("Expected key `target` to instantiate.")
    return get_obj_from_str(config["target"])(**config.get("params", dict()))

data = instantiate_from_config(config.data)
data.batch_size=4
data.prepare_data()
data.setup()

data_test =data._val_dataloader()
for batch in data_test:
    seqs = batch["seqs"]
    pdb = batch["pdb"]
    print("pdb",pdb)
    kd = batch["kd"].to(torch.float32).to(DEVICE)
    kon = batch["kon"].to(torch.float32).to(DEVICE)
    koff = batch["koff"].to(torch.float32).to(DEVICE)
    
    Ab_H = [" ".join(model.tokenizer.split(i)) for i in seqs["H"]]
    Ab_L = [" ".join(model.tokenizer.split(i)) for i in seqs["L"]]
    Ag_X = seqs["X"]

    outputs_pm = model(Ab_H,Ab_L,Ag_X, 0)
    kd_pre, kon_pre, koff_pre = model.loss.DT_pre(outputs_pm)

    # 这里同时计算所有的token进行损失计算
    loss_kd = model.loss.L1Loss(kd_pre, kd)
    loss_off = model.loss.huber_loss(koff_pre, koff) 
    loss_kon = model.loss.huber_loss((koff_pre-kd_pre), kon) 
        
    loss = loss_kd + loss_off 
    print("pre:",kd_pre.detach(),koff_pre.detach(),(koff_pre-kd_pre).detach())
    print("label:",kd.detach(),koff.detach(),kon.detach())
    print("loss:",loss_kd.detach(),loss_off.detach(),loss_kon.detach())
    print("__________________________________________________")
    # break



pdb ['4JPK_HL_A', '4JPK_HL_A', '4U6H_AB_E', '1MLC_AB_E']
pre: tensor([9.1051, 9.1014, 9.1713, 9.3624], device='cuda:0') tensor([3.6847, 3.6917, 3.6427, 3.3303], device='cuda:0') tensor([-5.4205, -5.4097, -5.5285, -6.0321], device='cuda:0')
label: tensor([ 6.9245,  6.8041, 10.0458, 10.2596], device='cuda:0') tensor([2.6615, 2.8041, 4.3188, 3.3316], device='cuda:0') tensor([-4.2625, -4.0000, -5.7097, -6.9243], device='cuda:0')
loss: tensor(1.0657, device='cuda:0') tensor(0.2864, device='cuda:0') tensor(0.4955, device='cuda:0')
__________________________________________________
pdb ['3NGB_HL_G', '3NGB_HL_G', '4JPK_HL_A', '3N85_A_LH']


RuntimeError: CUDA out of memory. Tried to allocate 508.00 MiB (GPU 0; 44.53 GiB total capacity; 42.81 GiB already allocated; 9.94 MiB free; 43.42 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF